# FlexPower Quant Challenge

<mark>Diclaimer</mark>

- The initial code and logic were entirely developed by me. ChatGPT was later used to streamline code formatting and expedite the creation of polished, clean plots. (This sentence was also improved by ChatGPT)
- In order to keep the github repo small, some of the plots, especially for task 2.7 where saved in the directory `results` but were not commited. Please run the full notebook to reproduce the plots.

## Task 1

<mark>Add explanation of solution for task one (add link to onion architecture) and also how to setup fastAPI</mark>

## Task 2



### Setup

Package imports, file imports and renaming of columns.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
file_name = pd.ExcelFile("app/analysis_task_data.xlsx")

df = pd.read_excel(file_name, sheet_name=None)['DE_Wind_PV_Prices']

In [ ]:
column_mapping = {
    "hour": "hour",
    "time": "timestamp",
    "Wind Day Ahead Forecast [in MW]": "da_wind",
    "Wind Intraday Forecast [in MW]": "intra_wind",
    "PV Day Ahead Forecast [in MW]": "da_pv",
    "PV Intraday Forecast [in MW]": "intra_pv",
    "Day Ahead Price hourly [in EUR/MWh]": "da_price",
    "Intraday Price Price Quarter Hourly  [in EUR/MWh]": "intra_price_15m",
    "Intraday Price Hourly  [in EUR/MWh]": "intra_price_1h",
    "Imbalance Price Quarter Hourly  [in EUR/MWh]": "imbalance_price_15m",
}

In [ ]:
df.rename(columns=column_mapping, inplace=True)
df.set_index('timestamp', inplace=True)
df.drop(columns=["hour"], inplace=True)
df = df.sort_index()

### Task 2.0
Initial overview over the given dataset. Plotting every time series.

In [ ]:
import matplotlib.pyplot as plt

# Set up the subplots
n_plots = len(df.columns)
fig, axes = plt.subplots(n_plots, 1, figsize=(12, n_plots * 2), sharex=True)

# Plot each column
for ax, column in zip(axes, df.columns):
    ax.plot(df.index, df[column], label=column)
    ax.set_ylabel(column)
    ax.grid()
    ax.legend(loc="upper right")

# Set common x-axis label and adjust layout
axes[-1].set_xlabel('Time')
plt.tight_layout()
plt.show()


### Task 2.1

How much Wind/PV Power was forecasted to produced in German in 2021 [in MWh] on Day Ahead (da) and on Intraday (id). Hint: Be careful: you have values in MW on a quarter hourly basis, think how this translates into hourly values.

**Answer + Notes:** 
- Since MW on a quarter hour basis, dividing by 4

In [ ]:
categories = ['Wind', 'PV']

# sum and convert into TWh
day_ahead_gen_forecast = [
    float(df['da_wind'].sum()) / 4 / 1_000_000,
    float(df['da_pv'].sum()) / 4 / 1_000_000,
]
intraday_gen_forecast = [
    float(df['intra_wind'].sum()) / 4 / 1_000_000,
    float(df['intra_pv'].sum()) / 4 / 1_000_000,
]

total_day_ahead = sum(day_ahead_gen_forecast)
total_intraday = sum(intraday_gen_forecast)

print(f"Day-Ahead Wind Forecasted Generation  : {day_ahead_gen_forecast[0]:>7.2f} [TWh]")
print(f"Intraday Wind Forecasted Generation   : {intraday_gen_forecast[0]:>7.2f} [TWh]")
print(f"Day-Ahead PV Forecasted Generation    : {day_ahead_gen_forecast[1]:>7.2f} [TWh]")
print(f"Intraday PV Forecasted Generation     : {intraday_gen_forecast[1]:>7.2f} [TWh]")
print(f"Total Day-Ahead Forecasted Generation : {total_day_ahead:>7.2f} [TWh]")
print(f"Total Intraday Forecasted Generation  : {total_intraday:>7.2f} [TWh]")

# Set up the bar plot
x = np.arange(len(categories))  # Label locations
width = 0.35  # Width of the bars

# Create the plot
fig, ax = plt.subplots(figsize=(9, 5))
bars1 = ax.bar(x - width/2, day_ahead_gen_forecast, width, label='Day-Ahead')
bars2 = ax.bar(x + width/2, intraday_gen_forecast, width, label='Intraday')

# Add labels, title, and legend
ax.set_title('Forecasted Day-Ahead vs Intraday Generation')
ax.set_xlabel('Generation Type')
ax.set_ylabel('Forecasted Generation [TWh]')
ax.set_ylim(35)
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

for bar in bars1:
    ax.text(
        bar.get_x() + bar.get_width() / 2, 
        bar.get_height() - 1,  # Position slightly below the top of the bar
        f"{bar.get_height():.2f}", 
        ha='center', 
        va='top', 
        color='white', 
        fontweight='bold'
    )

for bar in bars2:
    ax.text(
        bar.get_x() + bar.get_width() / 2, 
        bar.get_height() - 1,  # Position slightly below the top of the bar
        f"{bar.get_height():.2f}", 
        ha='center', 
        va='top', 
        color='white', 
        fontweight='bold'
    )

# Show the plot
plt.tight_layout()
plt.show()


### Task 2.2

Show the average Wind/Solar production for 2021 over a 24h period for Intraday and Day Ahead (4 lines in one graph).

**Answer + Notes:** 
- Assuming mean for each hour of the day for the whole year
- Adding the same plot for the prices

In [ ]:
import matplotlib.pyplot as plt

# Calculate the mean for each hour across all days and convert to GW
df_hourly_avg = df[['da_wind', 'da_pv', 'intra_wind', 'intra_pv']].groupby(df.index.hour).mean() / 1_000

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
df_hourly_avg.plot(ax=ax)

# Set labels and title
ax.set_xlabel('Hour of the Day')
ax.set_ylabel('Average Forecasted Generation [GW]')
ax.set_title('Hourly Average Forecasted Generation for Wind and PV')
ax.grid(True)
ax.legend(title="Forecast Type")
ax.set_xticks(range(0, 24))  # Ensure one tick per hour
plt.xticks(rotation=0)  # Keep hour labels horizontal for readability

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calculate the mean for each hour across all days
df_hourly_avg = df[['da_price', 'intra_price_15m', 'intra_price_1h']].groupby(df.index.hour).mean()

# Plotting
fig, ax = plt.subplots(figsize=(10, 6))
df_hourly_avg.plot(ax=ax)

# Set labels and title
ax.set_xlabel('Hour of the Day')
ax.set_ylabel('Average Price [EUR/MWh]')
ax.set_title('Hourly Average Price Comparison: Day-Ahead vs Intraday')
ax.grid(True)
ax.legend(title="Price Type")
ax.set_xticks(range(0, 24))  # One tick per hour for clarity
plt.xticks(rotation=0)  # Keep hour labels horizontal

# Show the plot
plt.tight_layout()
plt.show()


### Task 2.3

What was the average value [in EUR/MWh] for Wind/Solar Power in 2021 using the da forecast and using da h prices? The average value is defined as the average hourly value that a Wind/PV farm owner would have received for their product. Is the average value of Wind and PV higher or lower than the average da price? Why could it be higher/lower?

**Answer + Notes:** 

Using the following formular:
$$\frac{\sum{(generation [15min] * price)}}{\sum{generation [15min]}}$$
The divivsion by 4 is not necessary.

In [ ]:
# Calculate the mean and weighted average values
mean_da_price = df['da_price'].mean()
weighted_avg_wind_price = (df['da_wind'] * df['da_price']).sum() / df['da_wind'].sum()
weighted_avg_pv_price = (df['da_pv'] * df['da_price']).sum() / df['da_pv'].sum()

# Print the results with consistent alignment and clear messages
print(f"{'Weighted Average Day-Ahead Price (Wind):':<40} {weighted_avg_wind_price:.2f} EUR/MWh")
print(f"{'Weighted Average Day-Ahead Price (PV):':<40} {weighted_avg_pv_price:.2f} EUR/MWh")
print(f"{'Overall Average Day-Ahead Price:':<40} {mean_da_price:.2f} EUR/MWh")

<mark>Add section about merit order model and why renewables produce lower prides</mark>

### Task 2.4

Find the Day with the highest renewable energy production and with the lowest renewable energy production in 2021. What was the average Day Ahead Price levels on these days? How do you explain the difference in prices?

In [ ]:
# Calculate the daily renewable generation for day-ahead
df['da_daily_renew'] = (df['da_wind'] + df['da_pv']).resample('D').sum()

# Get timestamps for the maximum and minimum renewable generation days
max_renew_timestamp = df['da_daily_renew'].idxmax()
min_renew_timestamp = df['da_daily_renew'].idxmin()

# Retrieve the total renewable generation for max and min days
max_renew_value = df['da_daily_renew'][max_renew_timestamp]
min_renew_value = df['da_daily_renew'][min_renew_timestamp]

# Filter day-ahead prices for max and min renewable generation days
df_max_renew_day_price = df[df.index.date == max_renew_timestamp.date()]['da_price']
df_min_renew_day_price = df[df.index.date == min_renew_timestamp.date()]['da_price']

# Calculate mean prices
max_renew_mean_price = df_max_renew_day_price.sum() / 4 / 24
min_renew_mean_price = df_min_renew_day_price.sum() / 4 / 24

# Print results with timestamps and values
print(f"Max Renewable Generation Day: {max_renew_timestamp.date()} with {max_renew_value:.2f} MWh")
print(f"Min Renewable Generation Day: {min_renew_timestamp.date()} with {min_renew_value:.2f} MWh")
print(f"Average Day-Ahead Price on Max Renewable Day: {max_renew_mean_price:8.2f} EUR/MWh")
print(f"Average Day-Ahead Price on Min Renewable Day: {min_renew_mean_price:8.2f} EUR/MWh")

<mark>Add section about merit order model and why renewables produce lower prides</mark>

### Task 2.5

What is the average hourly da price during week days vs during weekends. Why do you think average prices may differ?

In [ ]:
# Identify weekends in the DataFrame
df['is_weekend'] = df.index.weekday >= 5

# Calculate mean day-ahead prices for weekdays and weekends
mean_weekday_price = df.loc[~df['is_weekend'], 'da_price'].mean()
mean_weekend_price = df.loc[df['is_weekend'], 'da_price'].mean()

# Print the results with clear labels
print(f"Mean Day-Ahead Price on Weekdays: {mean_weekday_price:.2f} EUR/MWh")
print(f"Mean Day-Ahead Price on Weekends: {mean_weekend_price:.2f} EUR/MWh")


<mark>Section about load being lower on weekend + maybe get load for 2021 from entso-e

### Task 2.6

How much revenue would you generate with a battery with a capacity of 1 MWh which you can fully charge and fully discharge (1 Cycle) every day in 2021? Think about when you would charge and when you would discharge and apply this rule for each day of the year.

**Answer + Notes:** 
- Looking at the average hourly prices already plotted in task 2.2, there a obvious price peaks at peak load. The evening load being higher. The lowest load is usually during the night, which is why prices always tend to be low during the night. Taking the minumum average hourly price to buy and the maximum average hourly price to sell.

In [ ]:
# Determine the optimal buy and sell hours based on average day-ahead price
buy_hour = int(df['da_price'].groupby(df.index.hour).mean().idxmin())
sell_hour = int(df['da_price'].groupby(df.index.hour).mean().idxmax())

# Filter the DataFrame for entries at the identified buy and sell hours
df_filtered = df[(df.index.hour == buy_hour) | (df.index.hour == sell_hour)][['da_price']]

# Pivot the DataFrame to have separate columns for buy and sell prices
df_pivoted = df_filtered.pivot_table(index=df_filtered.index.date, columns=df_filtered.index.hour, values='da_price')

# Rename columns to clearly indicate buy and sell prices, regardless of order
df_pivoted = df_pivoted.rename(columns={buy_hour: 'price_buy', sell_hour: 'price_sell'})

# Drop rows with missing values (in case any timestamps are missing)
df_pivoted = df_pivoted.dropna()

# Calculate the daily price difference (price_sell - price_buy)
df_pivoted['price_difference'] = df_pivoted['price_sell'] - df_pivoted['price_buy']

# Display the result
print(f"Optimal Buy Hour: {buy_hour:02d}:00, Sell Hour: {sell_hour:02d}:00")
print(f"Total Price Difference Sum: {df_pivoted['price_difference'].sum():.2f} EUR")


### Task 2.7

Come up with a trading strategy that makes money between the day ahead hourly prices and the intraday hourly prices. A strategy could be something like, always buy hour 19-20 on day ahead and sell it on intraday. You can look at certain times, weekdays, seasons, production levels of wind and solar. Your strategy can have a few input paramters such as time, renewable production etc, and then a decision output between two prices. I.e. when do you want to go long and short. Show the cumulative performance of this strategy with a 100 MW position. Show your results and quickly explain your reasoning of why you think this strategy might be a good idea and why it does or does not work.

**Answer + Notes:**
- Most plots are saved into the directory `results`, which is not commited to keep the repo size small. Please run the notebook to generate all plots.

In [ ]:
import matplotlib.pyplot as plt

# Add a weekday/weekend indicator
df['is_weekend'] = df.index.to_series().apply(lambda x: x.weekday() >= 5)

# Separate data for weekdays and weekends
weekday_data = df[df['is_weekend'] == False]
weekend_data = df[df['is_weekend'] == True]

# Group by hour and calculate mean for each hour
weekday_hourly_avg = weekday_data[['da_price', 'intra_price_15m', 'intra_price_1h']].groupby(weekday_data.index.hour).mean()
weekend_hourly_avg = weekend_data[['da_price', 'intra_price_15m', 'intra_price_1h']].groupby(weekend_data.index.hour).mean()

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Weekday plot
weekday_hourly_avg.plot(kind='line', ax=ax1)
ax1.set_title('Average Prices by Hour of the Day (Weekdays)')
ax1.set_ylabel('Average Value')
ax1.legend(title="Columns")
ax1.grid()

# Weekend plot
weekend_hourly_avg.plot(kind='line', ax=ax2)
ax2.set_title('Average Prices by Hour of the Day (Weekends)')
ax2.set_xlabel('Hour of the Day')
ax2.set_ylabel('Average Value')
ax2.legend(title="Columns")
ax2.grid()

plt.xticks(rotation=0)  # Keeps the hour labels horizontal
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Calculate the differences
df['diff_price_15m'] = df['da_price'] - df['intra_price_15m']
df['diff_price_1h'] = df['da_price'] - df['intra_price_1h']

# Add a weekday/weekend indicator
df['is_weekend'] = df.index.to_series().apply(lambda x: x.weekday() >= 5)

# Separate data for weekdays and weekends
weekday_data = df[df['is_weekend'] == False]
weekend_data = df[df['is_weekend'] == True]

# Group by hour and calculate mean for each hour
weekday_diff_avg = weekday_data[['diff_price_15m', 'diff_price_1h']].groupby(weekday_data.index.hour).mean()
weekend_diff_avg = weekend_data[['diff_price_15m', 'diff_price_1h']].groupby(weekend_data.index.hour).mean()

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Weekday plot
weekday_diff_avg.plot(kind='line', ax=ax1)
ax1.set_title('Difference of da_price to Other Prices by Hour of the Day (Weekdays)')
ax1.set_ylabel('Difference Value')
ax1.legend(title="Differences")
ax1.grid()

# Weekend plot
weekend_diff_avg.plot(kind='line', ax=ax2)
ax2.set_title('Difference of da_price to Other Prices by Hour of the Day (Weekends)')
ax2.set_xlabel('Hour of the Day')
ax2.set_ylabel('Difference Value')
ax2.legend(title="Differences")
ax2.grid()

plt.xticks(rotation=0)  # Keeps the hour labels horizontal
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import calendar
import os

# Ensure the output directory exists
output_dir = "/app/results/price_diff_hours"
os.makedirs(output_dir, exist_ok=True)

# Calculate the difference as per the new line
df['diff_price_1h'] = (df['intra_price_1h'] - df['da_price'])

# Add a weekday/weekend indicator
df['is_weekend'] = df.index.to_series().apply(lambda x: x.weekday() >= 5)

# Loop through each month
for month in range(1, 13):
    # Filter data for the current month
    monthly_data = df[df.index.month == month]
    
    # Separate weekday and weekend data for the current month
    weekday_data = monthly_data[monthly_data['is_weekend'] == False]
    weekend_data = monthly_data[monthly_data['is_weekend'] == True]
    
    # Group by hour and calculate mean for each hour
    weekday_diff_avg = weekday_data['diff_price_1h'].groupby(weekday_data.index.hour).mean()
    weekend_diff_avg = weekend_data['diff_price_1h'].groupby(weekend_data.index.hour).mean()
    
    # Calculate maximum values for horizontal lines
    weekday_max = weekday_diff_avg.max()
    weekend_max = weekend_diff_avg.max()
    
    # Plotting
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
    
    # Weekday plot
    weekday_diff_avg.plot(kind='line', ax=ax1)
    ax1.axhline(y=weekday_max, color='red', linestyle='--', label=f'Max Profit: {weekday_max:.2f}')
    ax1.set_title(f'{calendar.month_name[month]} - Adjusted Profit by Hour (Weekdays)')
    ax1.set_ylabel('Profit Value')
    ax1.grid()
    ax1.legend()
    
    # Weekend plot
    weekend_diff_avg.plot(kind='line', ax=ax2)
    ax2.axhline(y=weekend_max, color='red', linestyle='--', label=f'Max Profit: {weekend_max:.2f}')
    ax2.set_title(f'{calendar.month_name[month]} - Adjusted Profit by Hour (Weekends)')
    ax2.set_xlabel('Hour of the Day')
    ax2.set_ylabel('Profit Value')
    ax2.grid()
    ax2.legend()
    
    plt.xticks(rotation=0)  # Keeps the hour labels horizontal
    plt.tight_layout()
    
    # Save the plot
    output_path = os.path.join(output_dir, f'{month:02d}_price_diff_hours_{calendar.month_name[month]}.png')
    plt.savefig(output_path)
    plt.close(fig)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# Ensure the output directory exists
output_dir = "/app/results/price_diff_gen_binned"
os.makedirs(output_dir, exist_ok=True)

# Calculate the difference for 1h only
df['diff_price_1h'] = df['intra_price_1h'] - df['da_price']

# Define bin edges (e.g., 0, 1000, 2000, ...)
bin_edges = np.arange(0, df[['da_wind', 'da_pv']].max().max() + 1000, 1000)

# Bin and calculate mean for each chunk for da_wind
df['wind_bin'] = pd.cut(df['da_wind'], bins=bin_edges)
wind_means_1h = df.groupby('wind_bin', observed=True)['diff_price_1h'].mean()

# Filter out bins with no data
wind_bin_centers = [interval.mid for interval, mean in zip(wind_means_1h.index, wind_means_1h) if not np.isnan(mean)]
wind_means_1h = wind_means_1h.dropna()

# Bin and calculate mean for each chunk for da_pv
df['pv_bin'] = pd.cut(df['da_pv'], bins=bin_edges)
pv_means_1h = df.groupby('pv_bin', observed=True)['diff_price_1h'].mean()

# Filter out bins with no data
pv_bin_centers = [interval.mid for interval, mean in zip(pv_means_1h.index, pv_means_1h) if not np.isnan(mean)]
pv_means_1h = pv_means_1h.dropna()

# Plotting for da_wind
fig, ax1 = plt.subplots(1, 1, figsize=(10, 6))

# Line plot for da_wind vs. mean(diff_price_1h)
ax1.plot(wind_bin_centers, wind_means_1h)
ax1.set_xlabel('da_wind (binned in chunks of 1000)')
ax1.set_ylabel('Mean Difference (1h)')
ax1.set_title('Mean Price Difference vs. Binned da_wind')
ax1.grid()

# Save the plot
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "price_diff_wind_binned.png"))
plt.close(fig)

# Plotting for da_pv
fig, ax2 = plt.subplots(1, 1, figsize=(10, 6))

# Line plot for da_pv vs. mean(diff_price_1h)
ax2.plot(pv_bin_centers, pv_means_1h)
ax2.set_xlabel('da_pv (binned in chunks of 1000)')
ax2.set_ylabel('Mean Difference (1h)')
ax2.set_title('Mean Price Difference vs. Binned da_pv')
ax2.grid()

# Save the plot
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "price_diff_pv_binned.png"))
plt.close(fig)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Calculate the differences
df['diff_price_15m'] = df['da_price'] - df['intra_price_15m']
df['diff_price_1h'] = df['da_price'] - df['intra_price_1h']

# Define bin edges for da_wind and da_pv
wind_bins = np.arange(0, df['da_wind'].max() + 1000, 1000)
pv_bins = np.arange(0, df['da_pv'].max() + 1000, 1000)

# Bin da_wind and da_pv
df['wind_bin'] = pd.cut(df['da_wind'], bins=wind_bins, labels=wind_bins[:-1])
df['pv_bin'] = pd.cut(df['da_pv'], bins=pv_bins, labels=pv_bins[:-1])

# Group by the bins and calculate mean difference for each bin
heatmap_data_15m = df.groupby(['wind_bin', 'pv_bin'])['diff_price_15m'].mean().unstack()
heatmap_data_1h = df.groupby(['wind_bin', 'pv_bin'])['diff_price_1h'].mean().unstack()

# Plotting the heatmaps using Matplotlib
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Heatmap for diff_price_15m
c1 = ax1.imshow(heatmap_data_15m, cmap='coolwarm', origin='lower', aspect='auto')
ax1.set_title('Heatmap of Price Difference (15m)')
ax1.set_xlabel('da_pv (binned)')
ax1.set_ylabel('da_wind (binned)')
fig.colorbar(c1, ax=ax1, label='Price Difference (15m)')

# Heatmap for diff_price_1h
c2 = ax2.imshow(heatmap_data_1h, cmap='coolwarm', origin='lower', aspect='auto')
ax2.set_title('Heatmap of Price Difference (1h)')
ax2.set_xlabel('da_pv (binned)')
ax2.set_ylabel('da_wind (binned)')
fig.colorbar(c2, ax=ax2, label='Price Difference (1h)')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Calculate the differences
df['diff_price_15m'] = df['da_price'] - df['intra_price_15m']
df['diff_price_1h'] = df['da_price'] - df['intra_price_1h']

# Define bin edges for da_wind and da_pv
wind_bins = np.arange(0, df['da_wind'].max() + 1000, 1000)
pv_bins = np.arange(0, df['da_pv'].max() + 1000, 1000)

# Loop through each hour of the day to create separate heatmaps
for hour in range(24):
    # Filter data for the current hour
    hourly_data = df[df.index.hour == hour].copy()  # Copy to avoid SettingWithCopyWarning
    
    # Bin da_wind and da_pv for the current hour's data
    hourly_data.loc[:, 'wind_bin'] = pd.cut(hourly_data['da_wind'], bins=wind_bins, labels=wind_bins[:-1])
    hourly_data.loc[:, 'pv_bin'] = pd.cut(hourly_data['da_pv'], bins=pv_bins, labels=pv_bins[:-1])
    
    # Group by the bins and calculate mean difference for each bin
    heatmap_data_15m = hourly_data.groupby(['wind_bin', 'pv_bin'], observed=True)['diff_price_15m'].mean().unstack()
    heatmap_data_1h = hourly_data.groupby(['wind_bin', 'pv_bin'], observed=True)['diff_price_1h'].mean().unstack()

    # Check if the heatmap data is empty, and skip plotting if so
    if heatmap_data_15m.empty or heatmap_data_1h.empty:
        print(f"Skipping hour {hour} due to lack of data.")
        continue

    # Plotting the heatmaps for the current hour
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Heatmap for diff_price_15m
    c1 = ax1.imshow(heatmap_data_15m, cmap='coolwarm', origin='lower', aspect='auto')
    ax1.set_title(f'Heatmap of Price Difference (15m) at Hour {hour}')
    ax1.set_xlabel('da_pv (binned)')
    ax1.set_ylabel('da_wind (binned)')
    fig.colorbar(c1, ax=ax1, label='Price Difference (15m)')

    # Heatmap for diff_price_1h
    c2 = ax2.imshow(heatmap_data_1h, cmap='coolwarm', origin='lower', aspect='auto')
    ax2.set_title(f'Heatmap of Price Difference (1h) at Hour {hour}')
    ax2.set_xlabel('da_pv (binned)')
    ax2.set_ylabel('da_wind (binned)')
    fig.colorbar(c2, ax=ax2, label='Price Difference (1h)')

    plt.tight_layout()
    plt.show()